In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [ ]:
! pip install datasets transformers sacrebleu

     |████████████████████████████████| 245kB 7.9MB/s 
     |████████████████████████████████| 2.5MB 49.9MB/s 
     |████████████████████████████████| 61kB 9.4MB/s 
     |████████████████████████████████| 122kB 59.6MB/s 
     |████████████████████████████████| 245kB 48.5MB/s 
     |████████████████████████████████| 3.3MB 27.2MB/s 
     |████████████████████████████████| 901kB 46.4MB/s 
ERROR: transformers 4.8.2 has requirement huggingface-hub==0.0.12, but you'll have huggingface-hub 0.0.13 which is incompatible.


In [ ]:
model_checkpoint = "Helsinki-NLP/opus-mt-tr-en"

In [ ]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("wmt16", "tr-en")
metric = load_metric("sacrebleu")

Dataset wmt16 downloaded and prepared to /root/.cache/huggingface/datasets/wmt16/tr-en/1.0.0/0d9fb3e814712c785176ad8cdb9f465fbe6479000ee6546725db30ad8a8b5f8a. Subsequent calls will reuse this data.


In [ ]:
! pip install sentencepiece

     |████████████████████████████████| 1.2MB 8.5MB/s 


In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
if "mbart" in model_checkpoint:
    tokenizer.src_lang = "tr-TR"
    tokenizer.tgt_lang = "en-XX"

In [ ]:
if model_checkpoint in ["t5-base"]:
    prefix = "translate Turkish to English: "
else:
    prefix = ""

In [ ]:
max_input_length = 128
max_target_length = 128
source_lang = "tr"
target_lang = "en"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
batch_size = 16
args = Seq2SeqTrainingArguments(
    "test-translation",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp fp16 backend


In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation.
***** Running training *****
  Num examples = 205756
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 25720


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.077300,1.652053,27.345000,24.301700
2,0.985100,1.660793,27.712800,24.371600


Saving model checkpoint to test-translation/checkpoint-500
Configuration saved in test-translation/checkpoint-500/config.json
Model weights saved in test-translation/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-translation/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-translation/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-translation/checkpoint-1000
Configuration saved in test-translation/checkpoint-1000/config.json
Model weights saved in test-translation/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-translation/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-translation/checkpoint-1000/special_tokens_map.json
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1299: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite

TrainOutput(global_step=25720, training_loss=1.0655269177853597, metrics={'train_runtime': 4171.699, 'train_samples_per_second': 98.644, 'train_steps_per_second': 6.165, 'total_flos': 2.353213317757747e+16, 'train_loss': 1.0655269177853597, 'epoch': 2.0})

In [ ]:
model.save_pretrained("/content/drive/My Drive/News article summarizer/Tr_En")
tokenizer.save_pretrained("/content/drive/My Drive/News article summarizer/Tr_En")

Configuration saved in /content/drive/My Drive/News article summarizer/Tr_En/config.json
Model weights saved in /content/drive/My Drive/News article summarizer/Tr_En/pytorch_model.bin
tokenizer config file saved in /content/drive/My Drive/News article summarizer/Tr_En/tokenizer_config.json
Special tokens file saved in /content/drive/My Drive/News article summarizer/Tr_En/special_tokens_map.json


('/content/drive/My Drive/News article summarizer/Tr_En/tokenizer_config.json',
 '/content/drive/My Drive/News article summarizer/Tr_En/special_tokens_map.json',
 PosixPath('/content/drive/My Drive/News article summarizer/Tr_En/source_spm'),
 PosixPath('/content/drive/My Drive/News article summarizer/Tr_En/target_spm'),
 PosixPath('/content/drive/My Drive/News article summarizer/Tr_En/vocab'),
 PosixPath('/content/drive/My Drive/News article summarizer/Tr_En/tokenizer_config_file'),
 '/content/drive/My Drive/News article summarizer/Tr_En/added_tokens.json')